In [1]:
from google.colab import files
uploaded = files.upload()

Saving voice.csv to voice.csv


In [2]:
import statistics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn import svm
from sklearn import preprocessing
from sklearn import manifold, datasets
from sklearn.manifold import SpectralEmbedding
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# To choose DR Method, Uncomment the selection
# Here parameters are only optimized for LTSA method with KNN classifier

#DR = 'LE'
#DR = 'HLLE'
DR = 'LTSA'
#DR = None

# To choose Classification Method, Uncomment the selection

Classification_method = 'KNN'
#Classification_method = 'DT'
#Classification_method = 'SVM'
#Classification_method = None

In [4]:
# To choose the name of Output file
method = str(DR) + str('_') + str(Classification_method) + str('_')

name_of_file = 'voice.csv'

# df = pd.read_excel (name_of_file) # If file is in xlsx format
df = pd.read_csv(name_of_file)

In [5]:
# To remove column with high co-relation (95% Here)
corr_matrix = df.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
cols_to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [6]:
k = df.drop(cols_to_drop, axis=1)
df = k
k = 1

In [7]:
# To shuffle the data
df_1 = shuffle(df)
df = df_1

# separate data and class
y = df.iloc[:, -1]
x = df.drop(df.columns[-1], axis=1)

k_temp = x

In [9]:
# Min-Max Scaling of the data (Scale the values between 0 & 1)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
x = x_scaled

# To print the size of X and Y (class)
print(x.shape)
print(y.shape)

(3168, 17)
(3168,)


In [11]:
x

array([[0.70951987, 0.42499822, 0.79649696, ..., 0.31612903, 0.22659042,
        0.24651379],
       [0.72531949, 0.18271555, 0.71620207, ..., 0.26451613, 0.29842745,
        0.18215585],
       [0.55441741, 0.4057829 , 0.59049965, ..., 0.21505376, 0.04208363,
        0.56415591],
       ...,
       [0.74931796, 0.1241635 , 0.74777248, ..., 0.23010753, 0.06862044,
        0.34847004],
       [0.63768989, 0.13373415, 0.64562445, ..., 0.35053763, 0.29878485,
        0.10469291],
       [0.78303894, 0.20181153, 0.78287465, ..., 0.04086021, 0.24088635,
        0.05088513]])

In [12]:
y

430       male
2173    female
701       male
1111      male
1730    female
         ...  
1914    female
1906    female
1948    female
1748    female
2202    female
Name: label, Length: 3168, dtype: object

In [13]:
main_accuracy = []

q_component = 0
# Loop to calculate optimised number of dimension using q_component
# Once the optimum dimension is found, Use N as a starting and N+1 as the ending value in for loop

for n_component in range(16, 17):

    q_component += 1
    print('Num of Component :: ', n_component)

    # LE - Locally Linear Embedding

    if (DR == 'LE'):
        clf = SpectralEmbedding(n_components=n_component, affinity='rbf', n_neighbors=300)
        X_le = clf.fit_transform(k_temp)
        x = X_le
        # print(x.shape)

    # HLLE - Hessian-based Locally Linear Embedding

    if (DR == 'HLLE'):
        hlle = manifold.LocallyLinearEmbedding(n_neighbors=300, n_components=n_component, eigen_solver='dense', method='hessian')
        X_hlle = hlle.fit_transform(k_temp)
        x = X_hlle
        # print(x.shape)

    # LTSA - Local Tangent Space Alignment

    if (DR == 'LTSA'):
        clf = manifold.LocallyLinearEmbedding(n_neighbors=300, n_components=n_component, eigen_solver='dense', method='ltsa')
        X_ltsa = clf.fit_transform(k_temp)
        x = X_ltsa
        # print(x.shape)

    # Train - Test Split
    # Here data is split in 80% training and 20% testing set
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

    # KNN - K NEAREST NEIGHBOUR

    if (Classification_method == 'KNN'):
        clf = KNeighborsClassifier(n_neighbors=3, algorithm='auto', leaf_size=3)
        nbc = clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

    # Decision Tree

    if (Classification_method == 'DT'):
        clf = DecisionTreeClassifier(criterion='entropy', random_state=1000)
        dt = clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)  # X_validation)

    # SVM - Support Vector Machine

    if (Classification_method == 'SVM'):
        clf = svm.SVC(kernel='rbf', decision_function_shape='ovr', gamma='scale', C=100)
        svm = clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)  # X_validation)

    # To us K-fold Cross-validation
    # Here K=10
    num_validations = 10

    accuracy = cross_val_score(clf, X_train, y_train, scoring = 'accuracy', cv = num_validations)
    main_accuracy.append(statistics.mean(accuracy))

Num of Component ::  16


In [17]:
# Print the overall accuracy
print(main_accuracy)

# To prepare Confusion Matrix
conf_mat_print = metrics.confusion_matrix(y_test, y_pred)

# TO draw Confusion Matrix
ax= plt.subplot()
sns.heatmap(conf_mat_print, annot=True, ax = ax, cmap='jet'); #annot=True to annotate cells
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['Male', 'Female']); ax.yaxis.set_ticklabels(['Male', 'Female']);

# Setting rest of the parameters to plot the Confusion matrix
name_out_graph = 'Final_Res' + '.jpg'
plt.rcParams.update({'font.size': 3})
f = plt.figure(1)
plt.savefig(name_out_graph, dpi=1000, fontsize=3, bbox_inches='tight')
plt.clf()


[0.977512371230276]


<Figure size 432x288 with 0 Axes>